In [1]:
import sys
sys.path.append("C:\\Users\\sefi\\Meta-Modelling\\Sample Size vs Accuracy\\Noise Level 3\\Rastrigin10D\\")
import Utils
import pyDOE
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

C:\Users\sefi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Helper Functions

In [2]:
''' Random Forest Implementation for Minimax Robustness'''
def rf_minimax(train_data,test_data,Hyper):
    depth, estim = Hyper
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:10].values, test_data.values])
    regr = RandomForestRegressor(max_depth=int(depth), random_state=0,n_estimators=int(estim)
                                ).fit(scaler.transform(train_data.iloc[:,:10]), train_data.iloc[:,11])
    pred = regr.predict(scaler.transform(test_data))
    return regr,pred

''' Random Forest Implementation for Composite Robustness'''
def rf_composite(train_data,test_data,Hyper):
    depth, estim = Hyper
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:10].values, test_data.values])
    regr = RandomForestRegressor(max_depth=int(depth), random_state=0,n_estimators=int(estim)
                                ).fit(scaler.transform(train_data.iloc[:,:10]), train_data.iloc[:,-1])
    pred = regr.predict(scaler.transform(test_data))
    return regr,pred

''' Latin HyperCube Sampling Design of Experiment for Hyper_Parameters Optimization'''
def DOE_Hyper():
    lhd = pyDOE.lhs(n=2, samples=100, criterion='m')
    X1, X2  = lhd[:,0], lhd[:,1] 
    X1 = Utils.linearscaletransform(X1,range_out=(3,50))
    X2 = Utils.linearscaletransform(X2,range_out=(5 , 100))
    Hyper_Parameters = pd.DataFrame()
    Hyper_Parameters['Max_Depth'] = pd.Series(X1)
    Hyper_Parameters['N_Estimators'] = pd.Series(X2)
    return Hyper_Parameters

## Load Training and Test Data Set initially Generated

## Random Forest Regression

In [3]:
X = [50,100,150,200,250,300,350,400,450,500]
Kriging_minimax = np.zeros([len(X),2])
Kriging_composite = np.zeros([len(X),2])
for k in range(len(X)):
    print (X[k])
    train = pd.read_csv(sys.path[1][:-2]+str('Training_Data_Sets\\'+str(X[k])+'Samples.csv')).iloc[:,1:]
    test = pd.read_csv(sys.path[1][:-2]+str('Test_Data_Sets\\750Samples.csv')).iloc[:,1:]
    true_minimax = np.array(test.True_Minimax)
    true_composite = np.array(test.True_Composite)
    Hyper_Parameters = DOE_Hyper()
    mean_abs_error_minimax = np.zeros(Hyper_Parameters.shape[0])
    mean_abs_error_composite = np.zeros(Hyper_Parameters.shape[0])
    for i in range(Hyper_Parameters.shape[0]):
        temp = np.array(Hyper_Parameters.iloc[i,:])
        model_minimax,pred_k_m = rf_minimax(train,test.iloc[:,:10],temp)
        model_composite,pred_k_c = rf_composite(train,test.iloc[:,:10],temp)
        mean_abs_error_minimax[i] = np.mean((abs(true_minimax-pred_k_m) / abs(true_minimax) ) * 100)
        mean_abs_error_composite[i] = np.mean((abs( true_composite-pred_k_c) / abs(true_composite) ) * 100)
    Hyper_Parameters ['Mean_Error_Minimax'] = pd.Series(mean_abs_error_minimax)
    Hyper_Parameters ['Mean_Error_Composite'] = pd.Series(mean_abs_error_composite)
    Kriging_minimax [k,:] = np.array(Hyper_Parameters.iloc[Hyper_Parameters.Mean_Error_Minimax.values.argmin(),:-2])
    Kriging_composite [k,:] = np.array(Hyper_Parameters.iloc[Hyper_Parameters.Mean_Error_Composite.values.argmin(),:-2])

50
100
150
200
250
300
350
400
450
500


In [4]:
print ('Best Configuration of Hyper Parameters for Robust Regularization')
Hyper_Parameters.iloc[Hyper_Parameters.Mean_Error_Minimax.values.argmin(),:-1]

Best Configuration of Hyper Parameters for Robust Regularization


Max_Depth             13.193233
N_Estimators          91.299099
Mean_Error_Minimax     2.101587
Name: 26, dtype: float64

In [5]:
print ('Best Configuration of Hyper Parameters for Robust Composition')
Hyper_Parameters.iloc[Hyper_Parameters.Mean_Error_Composite.values.argmin(),[0,1,3]]

Best Configuration of Hyper Parameters for Robust Composition


Max_Depth               37.557966
N_Estimators            77.571977
Mean_Error_Composite     1.426992
Name: 22, dtype: float64

In [6]:
Hyper_Parameters.to_csv('Result_Hyper_Params.csv')

In [7]:
Kriging_minimax

array([[ 29.69893932, 100.        ],
       [  8.56504336,  80.49438762],
       [ 28.8902804 ,  63.65197546],
       [ 47.74481803,  84.77656662],
       [ 36.46854626,  97.92610235],
       [ 38.54011526,  26.2453932 ],
       [ 20.34493906,  98.29468776],
       [ 44.0401464 ,  98.22548255],
       [ 12.0570569 , 100.        ],
       [ 13.1932333 ,  91.29909851]])

In [8]:
Kriging_composite

array([[ 29.69893932, 100.        ],
       [ 36.21916509,  48.17257983],
       [ 41.71332909,  99.42374502],
       [ 24.7565058 ,  94.18487902],
       [ 18.3381423 ,  65.67895851],
       [ 48.13912775,  61.51538399],
       [ 41.74403351,  95.59003012],
       [ 27.45050946,  87.57772678],
       [ 26.00001721,  86.27326451],
       [ 37.55796589,  77.57197732]])